In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

data_hist_files = {'A': 'data_file_hist_all_cols', 
                   'B': 'data_file_hist_some_cols',
                   'C': 'data_file_hist_no_lkbk'}

data = []
for c, fname in data_hist_files.items():
    df_hist = pd.read_pickle(f'data/{fname}.pkl')
    test_cols = [d for d in df_hist.columns if d.endswith('test_scores')]
    other_cols = ['pos_labels', 'sim_num_trades', 'sim_max_profit']
    df = df_hist[other_cols + test_cols].copy()
    
    # calculate true positives and false positives
    for i,r in df.iterrows():
        relevant_elements = r[0]
        trades = r[1]
        profit = r[2]
        models = list(r.index)[3:]
        for m in models:
            if pd.isnull(r[m]):
                continue
            d = json.loads(r[m].replace("'",'"'))
            tp = int(d['rc'] * relevant_elements)
            fp = int(tp/d['pr']-tp) if d['pr'] != 0 else 0
            ratio = round(tp/fp,2) if fp>0 else 0
            act_trades = min(trades,tp)
            row = dict(c=c,
                       dataset=i,
                       classifer=m,
                       tp=tp,
                       fp=fp,
                       diff=tp-fp,
                       ratio=ratio,
                       pr=d['pr'],
                       rc=d['rc'],
                       est_max_profit=(profit/trades)*act_trades if trades>0 else 0
                      )
            data.append(row)
df = pd.DataFrame(data).set_index(['c','dataset','classifer'])

In [2]:
rank1 = df['diff'].rank(ascending=False)            # Highest tp-fp difference
rank2 = df['pr'].rank(ascending=False)              # Highest precision
rank3 = df['ratio'].rank(ascending=False)           # Highest tp/fp ratio
rank4 = df['tp'].rank(ascending=False)              # Highest true positives
rank5 = df['est_max_profit'].rank(ascending=False)  # Highest estimated max profit

ranks = [rank1,rank2,rank3,rank4,rank5]

df['score'] = sum(ranks)/len(ranks)                 # Average of ranks
df.sort_values(by='score')

tp     fp   diff  ratio      pr      rc  \
c dataset   classifer                                                     
A 20210806i abc_test_scores   3534   3399    135   1.04  0.5097  0.0639   
B 20210806i gbc_test_scores   2710   2460    250   1.10  0.5241  0.0490   
A 20210806i lrc_test_scores   1731   1535    196   1.13  0.5299  0.0313   
            gbc_test_scores   1919   1770    149   1.08  0.5201  0.0347   
B 20210806i lrc_test_scores    840    631    209   1.33  0.5708  0.0152   
C 20210806i lrc_test_scores    453    277    176   1.64  0.6204  0.0082   
A 20210806g rfc_test_scores   2075   1856    219   1.12  0.5278  0.0656   
C 20210806i gbc_test_scores   3562   3670   -108   0.97  0.4925  0.0644   
            abc_test_scores   4259   4649   -390   0.92  0.4781  0.0770   
B 20210806i abc_test_scores   3429   3832   -403   0.89  0.4722  0.0620   
A 20210806d rfc_test_scores   2574   3207   -633   0.80  0.4452  0.0575   
C 20210806i gnb_test_scores  11134  12784  -1650   0.87  0.4655  0.2013   
  20210806d gbc_test_scores     58     38     20   1.53  0.6000  0.0013   
A 20210806i rfc_test_scores  12302  14848  -2546   0.83  0.4531  0.2224   
            xgb_test_scores   9746  11839  -2093   0.82  0.4515  0.1762   
C 20210806d mlp_test_scores   2963   3982  -1019   0.74  0.4266  0.0662   
B 20210806i rfc_test_scores  13895  17113  -3218   0.81  0.4481  0.2512   
C 20210806d lrc_test_scores     40     26     14   1.54  0.6000  0.0009   
  20210806i mlp_test_scores  10166  12541  -2375   0.81  0.4477  0.1838   
A 20210806a abc_test_scores     60     54      6   1.11  0.5263  0.0016   
  20210806g abc_test_scores     98     13     85   7.54  0.8824  0.0031   
  20210806d xgb_test_scores   2995   4185  -1190   0.72  0.4171  0.0669   
C 20210806i xgb_test_scores  11621  14481  -2860   0.80  0.4452  0.2101   
B 20210806i xgb_test_scores  13767  17107  -3340   0.80  0.4459  0.2489   
C 20210806i rfc_test_scores  14724  19186  -4462   0.77  0.4342  0.2662   
B 20210806g rfc_test_scores   3287   3631   -344   0.91  0.4751  0.1039   
A 20210806i mlp_test_scores  18757  25491  -6734   0.74  0.4239  0.3391   
C 20210806a mlp_test_scores   2449   3459  -1010   0.71  0.4145  0.0645   
B 20210806i gnb_test_scores  30871  40838  -9967   0.76  0.4305  0.5581   
C 20210806d xgb_test_scores   4861   7343  -2482   0.66  0.3983  0.1086   
B 20210806d rfc_test_scores   5072   7700  -2628   0.66  0.3971  0.1133   
A 20210806i gnb_test_scores  30699  41466 -10767   0.74  0.4254  0.5550   
B 20210806d xgb_test_scores   4588   7149  -2561   0.64  0.3909  0.1025   
A 20210806d lrc_test_scores    308    393    -85   0.78  0.4393  0.0069   
C 20210806d rfc_test_scores   5918   9240  -3322   0.64  0.3904  0.1322   
B 20210806i mlp_test_scores  11190  15541  -4351   0.72  0.4186  0.2023   
  20210806d gnb_test_scores  21842  36574 -14732   0.60  0.3739  0.4879   
  20210806h rfc_test_scores   1504   1852   -348   0.81  0.4481  0.0568   
A 20210806d mlp_test_scores  11599  19455  -7856   0.60  0.3735  0.2591   
            gnb_test_scores  25137  42782 -17645   0.59  0.3701  0.5615   
C 20210806a rfc_test_scores   5544   9030  -3486   0.61  0.3804  0.1460   
  20210806d gnb_test_scores   1875   3207  -1332   0.58  0.3689  0.0419   
B 20210806d mlp_test_scores   6025  10554  -4529   0.57  0.3634  0.1346   
            lrc_test_scores     98    124    -26   0.79  0.4412  0.0022   
  20210806a rfc_test_scores   5104   8657  -3553   0.59  0.3709  0.1344   
  20210806d gbc_test_scores     17     17      0   1.00  0.5000  0.0004   
C 20210806b mlp_test_scores    313    463   -150   0.68  0.4031  0.0132   
A 20210806a gnb_test_scores  16862  30094 -13232   0.56  0.3591  0.4440   
            rfc_test_scores   4116   7179  -3063   0.57  0.3644  0.1084   
  20210806h rfc_test_scores    813   1167   -354   0.70  0.4106  0.0307   
B 20210806a gnb_test_scores  14682  26501 -11819   0.55  0.3565  0.3866   
C 20210806a xgb_test_scores   4177   7409  -3232   0.56  

In [4]:
df_hist

,use_atr,ratio,reverse,window,length,pos_labels,imbalance,train_imbal,test_imbal,sim_init_val,sim_fee_buy,sim_fee_sell,sim_fee_per,sim_num_trades,sim_max_profit,sim_bad_trades,gnb_train_scores,gnb_test_scores,lrc_train_scores,lrc_test_scores,rfc_train_scores,rfc_test_scores,abc_train_scores,abc_test_scores,gbc_train_scores,gbc_test_scores,xgb_train_scores,xgb_test_scores,mlp_train_scores,mlp_test_scores
suffix,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20210806a,False,"(0.01, 0.005)",True,30,141174,37979,0.269023,0.262033,0.319201,1.0,0.0,0.0,0.1,1179.0,13228.993600,0.0,"{'f1': 0.0377, 'pr': 0.3764, 'rc': 0.0198}","{'f1': 0.0217, 'pr': 0.3077, 'rc': 0.0113}","{'f1': 0.0001, 'pr': 0.1818, 'rc': 0.0001}","{'f1': 0.0016, 'pr': 0.2083, 'rc': 0.0008}","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0}","{'f1': 0.211, 'pr': 0.3804, 'rc': 0.146}","{'f1': 0.0096, 'pr': 0.6683, 'rc': 0.0048}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.0153, 'pr': 0.9636, 'rc': 0.0077}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.588, 'pr': 0.9309, 'rc': 0.4298}","{'f1': 0.1685, 'pr': 0.3605, 'rc': 0.11}","{'f1': 0.1603, 'pr': 0.5985, 'rc': 0.0926}","{'f1': 0.1116, 'pr': 0.4145, 'rc': 0.0645}"
20210806b,False,"(0.01, 0.0025)",True,30,141174,23782,0.168459,0.169634,0.196954,1.0,0.0,0.0,0.1,1093.0,6620.481356,0.0,"{'f1': 0.0203, 'pr': 0.2266, 'rc': 0.0106}","{'f1': 0.0148, 'pr': 0.2586, 'rc': 0.0076}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0}","{'f1': 0.038, 'pr': 0.3008, 'rc': 0.0203}","{'f1': 0.0035, 'pr': 1.0, 'rc': 0.0017}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.0044, 'pr': 0.975, 'rc': 0.0022}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.3172, 'pr': 0.9575, 'rc': 0.1901}","{'f1': 0.0153, 'pr': 0.265, 'rc': 0.0079}","{'f1': 0.0469, 'pr': 0.5709, 'rc': 0.0245}","{'f1': 0.0255, 'pr': 0.4031, 'rc': 0.0132}"
20210806c,False,"(0.0075, 0.0025)",True,30,141174,29824,0.211257,0.220094,0.233808,1.0,0.0,0.0,0.1,1456.0,3126.592203,0.0,"{'f1': 0.2332, 'pr': 0.2506, 'rc': 0.218}","{'f1': 0.182, 'pr': 0.2573, 'rc': 0.1408}","{'f1': 0.0001, 'pr': 0.5, 'rc': 0.0}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0}","{'f1': 0.042, 'pr': 0.3302, 'rc': 0.0224}","{'f1': 0.005, 'pr': 0.8169, 'rc': 0.0025}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.0063, 'pr': 0.9481, 'rc': 0.0032}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.3233, 'pr': 0.9507, 'rc': 0.1948}","{'f1': 0.0222, 'pr': 0.2857, 'rc': 0.0115}","{'f1': 0.032, 'pr': 0.654, 'rc': 0.0164}","{'f1': 0.0117, 'pr': 0.28, 'rc': 0.006}"
20210806d,True,"(2, 1)",True,15,141174,44769,0.317119,0.317522,0.341973,1.0,0.0,0.0,0.1,1128.0,17342.974458,1.0,"{'f1': 0.0816, 'pr': 0.3498, 'rc': 0.0462}","{'f1': 0.0753, 'pr': 0.3689, 'rc': 0.0419}","{'f1': 0.0002, 'pr': 0.2308, 'rc': 0.0001}","{'f1': 0.0017, 'pr': 0.6, 'rc': 0.0009}","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0}","{'f1': 0.1975, 'pr': 0.3904, 'rc': 0.1322}","{'f1': 0.0081, 'pr': 0.6476, 'rc': 0.0041}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.0229, 'pr': 0.8178, 'rc': 0.0116}","{'f1': 0.0026, 'pr': 0.6, 'rc': 0.0013}","{'f1': 0.534, 'pr': 0.8965, 'rc': 0.3802}","{'f1': 0.1707, 'pr': 0.3983, 'rc': 0.1086}","{'f1': 0.219, 'pr': 0.5916, 'rc': 0.1344}","{'f1': 0.1145, 'pr': 0.4266, 'rc': 0.0662}"
20210806e,True,"(4, 2)",True,15,141174,17024,0.120589,0.123430,0.124245,1.0,0.0,0.0,0.1,335.0,416.560613,0.0,"{'f1': 0.0139, 'pr': 0.1601, 'rc': 0.0073}","{'f1': 0.0102, 'pr': 0.1857, 'rc': 0.0052}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0}","{'f1': 0.0125, 'pr': 0.2222, 'rc': 0.0064}","{'f1': 0.0017, 'pr': 0.7857, 'rc': 0.0009}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.0111, 'pr': 0.9863, 'rc': 0.0056}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0}","{'f1': 0.523, 'pr': 0.9794, 'rc': 0.3568}","{'f1': 0.0627, 'pr': 0.4421, 'rc': 0.0338}","{'f1': 0.0607, 'pr': 0.7057, 'rc': 0.0317}","{'f1': 0.0179, 'pr': 0.2805, 'rc': 0.0092}"
20210806f,True,"(4, 1)",True,